In [3]:
 pip install dask[dataframe]

     |████████████████████████████████| 118 kB 5.3 MB/s 


###importing necessary library

In [4]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import dask.dataframe as dd
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

###Read data with dask Dataframe

In [6]:
data =dd.read_csv("all-the-news-2-1.csv",error_bad_lines=False,engine='python',encoding="utf8"
                  ,dtype={'Unnamed: 0':'object','Unnamed: 0.1':'object','day':'object','year':'object','month':'object'})
data.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication
0,0,0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,1,1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2,2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,3,3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,4,4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


###Drop unnecessary columns

In [7]:
df=data.drop(['Unnamed: 0','Unnamed: 0.1','url','section','author','day','month','date','year','publication'],axis=1)

In [8]:
data=df.fillna(0)  ##Fill na with zero
data.dtypes

title      object
article    object
dtype: object

###combining the title and article column text

In [9]:
columns=data.columns
data['text']=data[columns].apply(func=(lambda row: ' '.join(row.values.astype(str))) ,axis=1,meta=('str'))
data=data.drop(columns,axis=1)

##**text preprocessing**

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
data['text'] = data['text'].apply(lambda x: word_tokenize(x),meta=('text'))

In [13]:
data.head()

,text
0,"[We, should, take, concerns, about, the, healt..."
1,"[Colts, GM, Ryan, Grigson, says, Andrew, Luck,..."
2,"[Trump, denies, report, he, ordered, Mueller, ..."
3,"[France, 's, Sarkozy, reveals, his, 'Passions,..."
4,"[Paris, Hilton, :, Woman, In, Black, For, Uncl..."


In [14]:
def normalize_tokens(list_of_tokens):
    return map(lambda x: x.lower(),list_of_tokens)

In [15]:
data['text'] = data['text'].apply(lambda x: normalize_tokens(x),meta=('text'))

In [16]:
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [17]:
data.head()

,text
0,"[we, should, take, concerns, about, the, healt..."
1,"[colts, gm, ryan, grigson, says, andrew, luck,..."
2,"[trump, denies, report, he, ordered, mueller, ..."
3,"[france, 's, sarkozy, reveals, his, 'passions,..."
4,"[paris, hilton, :, woman, in, black, for, uncl..."


In [18]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [19]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [20]:
def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [21]:
data['text'] = data['text'].apply(lambda x: contractions_expansion(x),meta=('text'))
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [22]:
data.head()

,text
0,"[we, should, take, concerns, about, the, healt..."
1,"[colts, gm, ryan, grigson, says, andrew, luck,..."
2,"[trump, denies, report, he, ordered, mueller, ..."
3,"[france, 's, sarkozy, reveals, his, 'passions,..."
4,"[paris, hilton, :, woman, in, black, for, uncl..."


In [23]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [24]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [25]:
def filter_waste_words(list_of_tokens):
    return filterfalse(waste_word_or_not,list_of_tokens)

In [26]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x),meta=('text'))
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [27]:
data.head()

,text
0,"[we, should, take, concerns, about, the, healt..."
1,"[colts, gm, ryan, grigson, says, andrew, luck,..."
2,"[trump, denies, report, he, ordered, mueller, ..."
3,"[france, sarkozy, reveals, his, but, insists, ..."
4,"[paris, hilton, woman, in, black, for, uncle, ..."


In [28]:
def split(list_of_tokens):
    return map(lambda x: re.split(regex,x)[0],list_of_tokens)

In [29]:
data['text'] = data['text'].apply(lambda x: split(x),meta=('text'))
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [30]:
data.head()

,text
0,"[we, should, take, concerns, about, the, healt..."
1,"[colts, gm, ryan, grigson, says, andrew, luck,..."
2,"[trump, denies, report, he, ordered, mueller, ..."
3,"[france, sarkozy, reveals, his, but, insists, ..."
4,"[paris, hilton, woman, in, black, for, uncle, ..."


In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [33]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [34]:
def stopwords_removal(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [35]:
data['text'] = data['text'].apply(lambda x: stopwords_removal(x),meta=('text'))
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [36]:
data.head()

,text
0,"[concerns, health, liberal, democracy, serious..."
1,"[colts, gm, ryan, grigson, says, andrew, luck,..."
2,"[trump, denies, report, ordered, mueller, fire..."
3,"[france, sarkozy, reveals, insists, cards, par..."
4,"[paris, hilton, woman, black, uncle, monty, fu..."


In [37]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    else:
        (treebank_tag[0],wordnet.NOUN)

In [38]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
def get_pos_tag(list_of_tokens):
    return map(get_wnet_pos_tag,pos_tag(list_of_tokens))

In [41]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
data['text'] = data['text'].apply(lambda x: get_pos_tag(x),meta=('text'))
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [43]:
data.head()

,text
0,"[(concerns, n), (health, n), (liberal, a), (de..."
1,"[(colts, n), (gm, v), (ryan, a), (grigson, n),..."
2,"[(trump, n), (denies, n), (report, n), (ordere..."
3,"[(france, n), (sarkozy, n), (reveals, n), (ins..."
4,"[(paris, a), (hilton, n), (woman, n), (black, ..."


In [44]:
 lemmatizer = WordNetLemmatizer()

In [45]:
def token_lemmatization(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [46]:
def lemmatization(list_of_tokens):
    if len(list_of_tokens) > 0:
        return map(lambda x: token_lemmatization(x),list_of_tokens)

In [47]:
data['text'] = data['text'].apply(lambda x: lemmatization(x),meta=('text'))

In [48]:
data['text'] = data['text'].apply(lambda x: list(x),meta=('text'))

In [49]:
data.head()

,text
0,"[concern, health, liberal, democracy, seriousl..."
1,"[colt, gm, ryan, grigson, say, , luck, contrac..."
2,"[trump, denies, report, order, mueller, fire, ..."
3,"[france, sarkozy, reveals, insist, card, paris..."
4,"[paris, hilton, woman, black, uncle, monty, fu..."


##**most frequent word for each row**

In [52]:
from collections import Counter

In [54]:
data['text'] = data['text'].apply(lambda x: [k for k, v in Counter(x).most_common(50)],meta=('text'))

In [55]:
data.head()

,text
0,"[democracy, heart, attack, seriously, foa, mou..."
1,"[colt, , luck, defense, contract, team, grigso..."
2,"[report, trump, order, fake, news, mueller, fi..."
3,"[sarkozy, return, party, president, politics, ..."
4,"[paris, monty, hilton, uncle, funeral, brinson..."
